In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/11/16 08:28:17 WARN Utils: Your hostname, ante-ThinkPad-T15-Gen-2i resolves to a loopback address: 127.0.1.1; using 192.168.6.93 instead (on interface wlp0s20f3)
23/11/16 08:28:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 08:28:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green_orig = spark.read.parquet('../../data/green/*')


df_green_repartitioned = df_green_orig.repartition(4)

df_green_repartitioned.write.parquet('../../data/pq/green', mode='overwrite')

In [4]:
df_yellow_orig = spark.read.parquet('../../data/yellow/*')


df_yellow_repartitioned = df_yellow_orig.repartition(4)

df_yellow_repartitioned.write.parquet('../../data/pq/yellow', mode='overwrite')

In [5]:
df_green = spark.read.parquet('../../data/pq/green/*')

In [6]:
df_green.registerTempTable('green')

/home/ante/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [9]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('HOUR', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/green', mode='overwrite')

In [11]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-01-09 18:00:00|  17|            125.25|             4|
|2021-01-15 16:00:00| 169|              21.8|             1|
|2021-01-21 20:00:00|  43|             60.65|             5|
|2021-01-25 17:00:00|  75|             344.0|            24|
|2021-01-27 13:00:00| 244|386.36000000000007|            14|
|2021-01-16 16:00:00|  95|              14.8|             1|
|2021-01-23 15:00:00|  52|             16.32|             2|
|2021-01-31 16:00:00|  82|              23.8|             1|
|2021-01-27 16:00:00|  97|253.90000000000003|            13|
|2021-01-16 17:00:00|  74|            123.97|            11|
|2021-01-16 22:00:00|  95|             37.36|             4|
|2021-01-15 14:00:00|  95|167.29999999999998|             9|
|2021-01-12 16:00:00| 197| 576.4499999999999|            12|
|2021-01-02 08:00:00| 25

In [12]:
df_yellow = spark.read.parquet('../../data/pq/yellow/*')
df_yellow.registerTempTable('yellow')

In [13]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-03 16:16:28|  2021-01-03 16:27:13|            1.0|          3.9|       1.0|                 N|         138|         179|           2|       13.0|  0.0|    0.5|       0.

In [14]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
GROUP BY
    1, 2
""")

In [15]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-01-17 10:00:00| 262|            515.81|            35|
|2021-01-08 14:00:00| 136|131.32999999999998|             3|
|2021-01-25 20:00:00| 236|            670.61|            43|
|2021-01-14 09:00:00| 186|1360.1600000000003|            98|
|2021-01-21 14:00:00| 233|           1013.47|            58|
|2021-01-05 16:00:00|  68| 834.6700000000001|            52|
|2021-01-28 20:00:00| 161|1166.8000000000002|            78|
|2021-01-26 11:00:00| 140|1933.4200000000003|           126|
|2021-01-16 15:00:00| 236|2077.9700000000007|           159|
|2021-01-02 17:00:00| 234|1595.1700000000003|            97|
|2021-01-25 08:00:00| 238|           2202.65|           135|
|2021-01-12 01:00:00|  74|             19.09|             2|
|2021-01-04 11:00:00| 164| 806.6500000000001|            55|
|2021-01-27 09:00:00| 23

In [16]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/yellow', mode='overwrite')

In [17]:
df_green_revenue = spark.read.parquet('../../data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../../data/report/revenue/yellow')

In [18]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [19]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [20]:
df_join.write.parquet('../../data/report/revenue/total', mode='overwrite')

In [21]:
df_join = spark.read.parquet('../../data/report/revenue/total')

In [22]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [23]:
df_zones = spark.read.parquet('../../zones/')

In [24]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [26]:
df_result.show()

+-------------------+----+------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2008-12-31 23:00:00| 193|        null|                null|               0.0|                    1|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|2009-01-01 00:00:00| 132|        null|                null|              14.3|                    1|       132|   Queens|         JFK Airport|    Airports|
|2009-01-01 00:00:00| 188|        null|                null|              10.3|                    1|       188| Brooklyn|Prospect-Lefferts...|   Boro Zone|
|2020-12-31 13:00:00| 170|        null|                nul

In [27]:
df_result.drop('LocationID', 'zone').write.parquet('../../tmp/revenue-zones', mode='overwrite')